In [ ]:
import json
import csv

In [ ]:
# code to line wrap the putput in this notebook
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
f = open('../aim5/Aim5_Data/neighbor-districts.json')

In [ ]:
data = json.load(f)
print(len(data))
data = dict((a.lower(),b) for a,b in data.items())
data2 = sorted(data)

723


**Solving District names Issues**

**Installing Libraries**

In [ ]:

!pip install fuzzywuzzy
!pip install python-Levenshtein

In [ ]:
from fuzzywuzzy import fuzz
import pandas as pd

repeat_element = set(['hamirpur', 'bilaspur', 'pratapgarh', 'balrampur', 'aurangabad'])

repeat_dict = {'hamirpur/Q2086180':'himachal pradesh','hamirpur/Q2019757':'uttar pradesh',
               'bilaspur/Q1478939':'himachal pradesh','bilaspur/Q100157':'chhattisgarh',
               'pratapgarh/Q1585433':'rajasthan','pratapgarh/Q1473962':'uttar pradesh',
               'balrampur/Q16056268':'chhattisgarh','balrampur/Q1948380':'uttar pradesh',
               'aurangabad/Q592942':'maharashtra','aurangabad/Q43086': 'bihar'}

repeat_dict1 = {'hamirpur':{'himachal pradesh':'hamirpur/Q2086180','uttar pradesh':'hamirpur/Q2019757'},
                'bilaspur':{'himachal pradesh':'bilaspur/Q1478939','chhattisgarh':'bilaspur/Q100157'},
                'pratapgarh':{'rajasthan':'pratapgarh/Q1585433','uttar pradesh':'pratapgarh/Q1473962'},
                'balrampur':{'chhattisgarh':'balrampur/Q16056268','uttar pradesh':'balrampur/Q1948380'},
                'aurangabad':{'maharashtra':'aurangabad/Q592942','bihar':'aurangabad/Q43086'}}
#repeat_element = set(repeat_element)

list1_neighbor = set([i.split('/')[0] for i in data2])
list1_neighbor.difference_update(repeat_element)



districts = pd.read_csv('../aim5/Aim5_Data/districts.csv')
list2 = set(districts['District'].unique())
list2 = set([i.lower() for i in list2])
list2.difference_update(repeat_element)

delhi_list = ['north_delhi', 'north_east_delhi', 'south_west_delhi', 'central_delhi', 'new_delhi', 
              'east_delhi', 'west_delhi', 'south_delhi', 'north_west_delhi', 'south_east_delhi',
              'shahdara']

list1_neighbor.difference_update(delhi_list)

list1_neighbor.add('delhi')

remove_list = ['airport quarantine','bsf camp','capf personnel','railway quarantine','evacuees','italians','other region','others','other state','unknown','foreign evacuees']

district_list_neighbor_map = {'aizawl' : ['aizwal'], 'banaskantha' : ['banas_kantha_district'], 'bengaluru rural' : ['bangalore_rural'], 'bengaluru urban' : ['bangalore_urban'], 'boudh' : ['baudh'], 'beed' : ['bid'], 'ferozepur' : ['firozpur'], 'hooghly' : ['hugli'], 'kancheepuram' : ['kanchipuram_district'], 'ganganagar' : ['sri_ganganagar'], 'koderma' : ['kodarma'], 'lahaul and spiti' : ['lahul_and_spiti'], 'navsari' : ['nav_sari_district'], 'narsinghpur' : ['narsimhapur'], 'shahid bhagat singh nagar' : ['shaheed_bhagat_singh_nagar'], 'thoothukkudi' : ['thoothukudi_district'], 'tumakuru' : ['tumkur_district'], 'rajouri' : ['rajauri'], 'rae bareli' : ['rae_bareilly'], 'sabarkantha' : ['sabar_kantha_district'], 'tiruchirappalli' : ['tiruchchirappalli_district'], 'sant kabir nagar' : ['sait_kibir_nagar'], 'yadgir' : ['yadagiri_district'], 'kaimur' : ['kaimur_bhabua'], 's.a.s. nagar' : ['sahibzada_ajit_singh_nagar'], 's.p.s. nellore' : ['sri_potti_sriramulu_nellore'], 'kabeerdham' : ['kabirdham'], 'dakshin bastar dantewada' : ['dantewada'], 'ballari' : ['bellary_district'], 'y.s.r. kadapa' : ['ysr'], 'sivaganga' : ['sivagangai_district'], 'east champaran' : ['purba_champaran'], 'belagavi' : ['belgaum_district'], 'nilgiris' : ['the_nilgiris_district'], 'devbhumi dwarka' : ['devbhumi_dwaraka_district'], 'tirunelveli' : ['tirunelveli_kattabo']}

for i in list2.intersection(list1_neighbor):
  district_list_neighbor_map[i] = [i]



district_list_neighbor_map.pop('kaimur')

list2.difference_update(set(remove_list))
list2.difference_update(set(district_list_neighbor_map.keys()))


list1_neighbor.difference_update(set([v[0] for k,v in district_list_neighbor_map.items()]))

temp = list2.intersection(list1_neighbor)

list2.difference_update(temp)
list1_neighbor.difference_update(temp)


#Dealing with '_' differnces in district names


for i in list1_neighbor:
  if (i.find('_') != -1 and i not in list2):
    j = i.replace('_',' ')
    if j in list2:
      list2.remove(j)
      list2.add(i)
      if j not in district_list_neighbor_map.keys():
        district_list_neighbor_map[j] = [i]
      else:
        district_list_neighbor_map[j].extend(i)


#print((sorted(district_list_neighbor_map.keys())))
#Dealing with '_district' differnces in district names
for i in list2:
  j = i.replace(' ','_')
  j = j+'_district'
  if j in list1_neighbor:
    list2.remove(i)
    list2.add(j)
    if i not in district_list_neighbor_map.keys():
        district_list_neighbor_map[i] = [j]
    else:
        district_list_neighbor_map[i].append(j)
    



#Dealing with Spelling mistakes
list3 = set([i for i in list2])


for i in list1_neighbor:
  for j in list3:
    if i!=j and fuzz.ratio(i,j) >= 88 and i not in list2 and i not in ['east_karbi_anglong','baleshwar']:
      #print(i+" "+j)
      list2.remove(j)
      list2.add(i)
      if j not in district_list_neighbor_map.keys():
        district_list_neighbor_map[j] = [i]
      else:
        district_list_neighbor_map[j].append(i)



district_neighbor = {'budgam': ['badgam'], 'vijayapura': ['bijapur_district'],'balasore':['baleshwar'],'dang':['the_dangs'], 
                     'chamarajanagara': ['chamarajanagar_district'], 'deogarh': ['debagarh'], 
                     'dholpur': ['dhaulpur'], 'karbi anglong': ['east_karbi_anglong'], 
                     'lakhimpur kheri': ['kheri'], 'mehsana': ['mahesana_district'], 
                     'sri muktsar sahib': ['muktsar'], 'panchmahal': ['panch_mahal_district'], 
                     'west champaran': ['pashchim_champaran'], 'west singhbhum': ['pashchimi_singhbhum'],
                     'shivamogga': ['shimoga_district'], 'tiruvannamalai': ['tiruvanamalai_district'],
                     'mumbai': ['mumbai_city', 'mumbai_suburban'],'east singhbhum' : ['purbi_singhbhum'],
                     'ayodhya' : ['faizabad'],'cooch behar':['kochbihar'],'amroha':['jyotiba_phule_nagar'],
                     'kaimur':['kaimur_(bhabua)'],'palakkad':['palghat'],'bhadohi':['sant_ravidas_nagar'],
                     'subarnapur':['sonapur']}



for i in district_neighbor:
  for j in district_neighbor[i]:
    list2.add(j)
    if i not in district_list_neighbor_map.keys():
        district_list_neighbor_map[i] = [j]
    else:
        district_list_neighbor_map[i].append(j)
  if i in list2:
    list2.remove(i)

#print(district_neighbor)


list_neighbor_district_map = {}
for i in district_list_neighbor_map:
  for j in district_list_neighbor_map[i]:
    list_neighbor_district_map[j] = i


#print(district_list_neighbor_map['bijapur'])


list3 = set(sorted(list2.symmetric_difference(list1_neighbor)))



print(len((district_list_neighbor_map)))
print(len(list_neighbor_district_map))
                                                                                                                                                                                                                                                                                                                                                                              
print((sorted(list3)))

#print((sorted(list2.symmetric_difference(list1_neighbor))))
print('shahdara' in list1_neighbor)
print('shahdara' in list2)

In [ ]:
# Finding new Districts from json files not present in districts.csv
li3 = []
for counter in list(range(1,17)):
  f = open('../aim5/Aim5_Data/aim5_jsons/raw_data{c}.json'.format(c=counter))
  #print(counter)
  data1 = json.load(f)

  for i in data1['raw_data']:
    a = i['detecteddistrict']
    b = i['detectedcity']
    if a!='':
      li3.append(a)
    elif b != '' :
      li3.append(b)
  



dis = [i.lower() for i in li3]
dis = set(dis)

districts = pd.read_csv('../aim5/Aim5_Data/districts.csv')
list2 = list(districts['District'])
list2 = set([i.lower() for i in list2])



dis.remove('.')
dis.remove('airport')
dis.remove('sarusajai quarantine centre')
dis.remove('unassigned')
dis.difference_update(list2)
#print(sorted(dis.difference(list2)))



temp = dis.intersection(list3)


json_list_neighbor_map = {}
for i in temp:
  json_list_neighbor_map[i] = [i]


dis.difference_update(temp)
list4 = list3.difference(temp)

temp1 = {'medchal malkajgiri' : ['medchal–malkajgiri'], 'komaram bheem' : ['komram_bheem'], 'ranga reddy' : ['rangareddy']
         ,'mancherial':['mancherial_district']}

for j in temp1:
  json_list_neighbor_map[j] = temp1[j]


for j in temp1:
  dis.remove(j)
  dis.add(temp1[j][0])
  list4.remove(temp1[j][0])
dis1 = set(sorted(dis))
print(sorted(list4))



['bhadradri_kothagudem', 'bishwanath', 'capital complex', 'chengalpattu', 'dima_hasao_district', 'east_sikkim_district', 'gaurela pendra marwahi', 'hnahthial', 'imphal_east', 'imphal_west', 'jayashankar_bhupalapally', 'jogulamba_gadwal', 'kamrup_metropolitan', 'khawzawl', 'konkan_division', 'marigaon', 'noklak', 'north_goa', 'north_sikkim_district', 'rajanna_sircilla', 'ranipet', 'saitual', 'south_goa', 'south_salmara-mankachar', 'south_sikkim_district', 'state pool', 'tenkasi', 'tirupathur', 'warangal_rural', 'warangal_urban', 'west_karbi_anglong', 'west_sikkim_district', 'yadadri_bhuvanagiri', 'yanam']


In [ ]:


for i in list4:

  if (i.find('_') != -1):
    j = i.replace('_',' ')
    if j in dis1:
      dis1.remove(j)
      dis1.add(i)
      if j not in json_list_neighbor_map.keys():
        json_list_neighbor_map[j] = [i]
      else:
        json_list_neighbor_map[j].extend(i)


list_neighbor_json_map = {}
for i in json_list_neighbor_map:
  for j in json_list_neighbor_map[i]:
    list_neighbor_json_map[j] = i


dis1 = dis1.difference(dis1.difference(list4))

print((sorted(list4.difference(dis1))))
print(len(json_list_neighbor_map))
#print(list_neighbor_json_map)

['bishwanath', 'capital complex', 'chengalpattu', 'dima_hasao_district', 'east_sikkim_district', 'gaurela pendra marwahi', 'hnahthial', 'khawzawl', 'konkan_division', 'marigaon', 'noklak', 'north_sikkim_district', 'ranipet', 'saitual', 'south_salmara-mankachar', 'south_sikkim_district', 'state pool', 'tenkasi', 'tirupathur', 'west_sikkim_district', 'yanam']
79


**Daily counts of each district 15march to 19April**



In [ ]:
import pandas as pd
import datetime
d1 = datetime.datetime(2020,3,15)
df = pd.read_csv('../aim5/Aim5_Data/15th_March_19th_April.csv')
df['Date Announced'] = df['Date Announced'].apply(lambda x: datetime.datetime.strptime(x,'%d/%m/%Y'))
df['Detected District'] = df['Detected District'].str.lower()
df['Detected State'] = df['Detected State'].str.lower()
df1 = df[['Date Announced','Detected District','Detected State']]
check = set(df1['Detected District'].unique())

#print(datetime.datetime.strptime('01/04/2020','%d/%m/%Y'))
#print(set(df1['Date Announced']))
i=0
for ind in df1.index:
  if df1['Date Announced'][ind] == d1:
    break
  i+=1

df1 = df1.iloc[i:]

#temp = df1.dropna()
#temp1 = df1.merge(temp, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']
#print(temp1)


dict1 = {}
## Making dictionary of dates and value as another dictionay of city and count 15 March to 19 April
for ind in df1.index:
  x = df1['Date Announced'][ind]
  y = df1['Detected District'][ind] 
  z = df1['Detected State'][ind]

  if pd.isna(z) and pd.isna(y):
    continue
  elif pd.isna(y) and not pd.isna(z):
    if z.find('delhi')!=-1:
      y = 'delhi'
    else:
      continue
  if y.find('delhi')!=-1:
    y = 'delhi'

  if  x not in dict1.keys():
    dict2 = {}
    if y in repeat_element:     #Working for same name districts
      key = repeat_dict1[y][z].lower()
      dict2[key] = 1
    else:
      dict2[y] = 1
    dict1[x] = dict2
  else:
    if y in repeat_element:     #Working for same name districts
      key = repeat_dict1[y][z].lower()
      if key not in dict1[x].keys():
        dict1[x][key]=1
      else:
        dict1[x][key] = dict1[x][key]+1
          
    elif y not in dict1[x].keys():
      dict1[x][y]=1
    else:
      dict1[x][y] = dict1[x][y]+1

print(dict1.keys())
#da = datetime.datetime(2020,3,2)
#print(da<d1)



dict_keys([Timestamp('2020-03-15 00:00:00'), Timestamp('2020-03-16 00:00:00'), Timestamp('2020-03-17 00:00:00'), Timestamp('2020-03-18 00:00:00'), Timestamp('2020-03-19 00:00:00'), Timestamp('2020-03-20 00:00:00'), Timestamp('2020-03-21 00:00:00'), Timestamp('2020-03-22 00:00:00'), Timestamp('2020-03-23 00:00:00'), Timestamp('2020-03-24 00:00:00'), Timestamp('2020-03-25 00:00:00'), Timestamp('2020-03-26 00:00:00'), Timestamp('2020-03-27 00:00:00'), Timestamp('2020-03-28 00:00:00'), Timestamp('2020-03-29 00:00:00'), Timestamp('2020-03-30 00:00:00'), Timestamp('2020-03-31 00:00:00'), Timestamp('2020-04-01 00:00:00'), Timestamp('2020-04-02 00:00:00'), Timestamp('2020-04-03 00:00:00'), Timestamp('2020-04-04 00:00:00'), Timestamp('2020-04-05 00:00:00'), Timestamp('2020-04-06 00:00:00'), Timestamp('2020-04-07 00:00:00'), Timestamp('2020-04-08 00:00:00'), Timestamp('2020-04-09 00:00:00'), Timestamp('2020-04-10 00:00:00'), Timestamp('2020-04-11 00:00:00'), Timestamp('2020-04-12 00:00:00'), Tim

*Including Json data for missing districts*


In [ ]:
f = open('../aim5/Aim5_Data/aim5_jsons/raw_data1.json')
data1 = json.load(f)


for i in data1['raw_data']:
  a = i['detecteddistrict'].lower()
  b = i['detectedcity'].lower()
  c = i['detectedstate'].lower()
  da = datetime.datetime.strptime(i['dateannounced'],'%d/%m/%Y')
  if da<d1 or i['currentstatus'] !='Hospitalized':
    continue
  
  if a!='' and a in json_list_neighbor_map.keys() and a not in check:
    
    print('reached here')
    print(a)
    if  da not in dict1.keys():
      dict2 = {}
      if a in repeat_element:     #Working for same name districts
        key = repeat_dict1[a][c].lower()
        dict2[key] = 1
      else:
        dict2[a] = 1
      dict1[da] = dict2
    else:
      if a in repeat_element:     #Working for same name districts
        key = repeat_dict1[a][c].lower()
        if key not in dict1[x].keys():
          dict1[x][key]=1
        else:
          dict1[x][key] = dict1[x][key]+1

      elif a not in dict1[da].keys():
        dict1[da][a]=1
      else:
        dict1[da][a] = dict1[da][a]+1
  elif b != '' and b in json_list_neighbor_map.keys() and b not in check:

    print('reached here1')
    print(b)
    if  da not in dict1.keys():
      dict2 = {}
      if b in repeat_element:     #Working for same name districts
        key = repeat_dict1[b][c].lower()
        dict2[key] = 1
      else:
        dict2[b] = 1
      dict1[da] = dict2
    else:
      if b in repeat_element:     #Working for same name districts
        key = repeat_dict1[b][c].lower()
        if key not in dict1[x].keys():
          dict1[x][key]=1
        else:
          dict1[x][key] = dict1[x][key]+1

      elif b not in dict1[da].keys():
        dict1[da][b]=1
      else:
        dict1[da][b] = dict1[da][b]+1

#print(dict1)

**Daily counts of each district 20April to 25April**

In [ ]:
d1 = datetime.datetime(2020,4,26)
df2 = pd.read_csv('../aim5/Aim5_Data/aim5_jsons/20th_April_26th_April.csv')
df2['Date Announced'] = df2['Date Announced'].apply(lambda x: datetime.datetime.strptime(x,'%d/%m/%Y'))
df2['Detected District'] = df2['Detected District'].str.lower()
df2['Detected State'] = df2['Detected State'].str.lower()
df2 = df2[['Date Announced','Detected District','Detected State']]

check = set(df2['Detected District'].unique())
i=0
for ind in df2.index:
  if df2['Date Announced'][ind] == d1:
    break
  i+=1

df2 = df2.iloc[:i]

#df2.dropna(inplace=True)
#temp = df2.dropna()
#temp1 = df2.merge(temp, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']

dict3 = {}
## Making dictionary of dates and value as another dictionay of city and count 20 April to 25 April
for ind in df2.index:
  x = df2['Date Announced'][ind]
  y = df2['Detected District'][ind] 
  z = df2['Detected State'][ind]

  if pd.isna(z) and pd.isna(y):
    continue
  elif pd.isna(y) and not pd.isna(z):
    if z.find('delhi')!=-1:
      y = 'delhi'
    else:
      continue
  if y.find('delhi')!=-1:
    y = 'delhi'

  if  x not in dict3.keys():
    dict4 = {}
    if y in repeat_element:     #Working for same name districts
      key = repeat_dict1[y][z].lower()
      dict4[key] = 1
    else:
      dict4[y] = 1
    dict3[x] = dict4
  else:
    if y in repeat_element:     #Working for same name districts
      key = repeat_dict1[y][z].lower()
      if key not in dict3[x].keys():
        dict3[x][key]=1
      else:
        dict3[x][key] = dict3[x][key]+1
          
    elif y not in dict3[x].keys():
      dict3[x][y]=1
    else:
      dict3[x][y] = dict3[x][y]+1


dict1.update(dict3)
print(dict1.keys())

dict_keys([Timestamp('2020-03-15 00:00:00'), Timestamp('2020-03-16 00:00:00'), Timestamp('2020-03-17 00:00:00'), Timestamp('2020-03-18 00:00:00'), Timestamp('2020-03-19 00:00:00'), Timestamp('2020-03-20 00:00:00'), Timestamp('2020-03-21 00:00:00'), Timestamp('2020-03-22 00:00:00'), Timestamp('2020-03-23 00:00:00'), Timestamp('2020-03-24 00:00:00'), Timestamp('2020-03-25 00:00:00'), Timestamp('2020-03-26 00:00:00'), Timestamp('2020-03-27 00:00:00'), Timestamp('2020-03-28 00:00:00'), Timestamp('2020-03-29 00:00:00'), Timestamp('2020-03-30 00:00:00'), Timestamp('2020-03-31 00:00:00'), Timestamp('2020-04-01 00:00:00'), Timestamp('2020-04-02 00:00:00'), Timestamp('2020-04-03 00:00:00'), Timestamp('2020-04-04 00:00:00'), Timestamp('2020-04-05 00:00:00'), Timestamp('2020-04-06 00:00:00'), Timestamp('2020-04-07 00:00:00'), Timestamp('2020-04-08 00:00:00'), Timestamp('2020-04-09 00:00:00'), Timestamp('2020-04-10 00:00:00'), Timestamp('2020-04-11 00:00:00'), Timestamp('2020-04-12 00:00:00'), Tim

**Total cases of each district till April 25**

In [ ]:
dict2 = {}
for i in dict1:
  for j in dict1[i]:
    if j not in dict2.keys():
      dict2[j] = dict1[i][j]
    else:
      dict2[j] = dict1[i][j]+dict2[j]
print(len(dict2))


433


**Generating cases 26th April onwards from district.csv**

In [ ]:
df3 = pd.read_csv('../aim5/Aim5_Data/districts.csv')
df3['Date'] = df3['Date'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d'))
df3['District'] = df3['District'].str.lower()
df3['State'] = df3['State'].str.lower()
df3 = df3[['Date','State','District','Confirmed']]
check = set(df3['District'].unique())

for ind in df3.index:
  
  w = df3['State'][ind]
  x = df3['Date'][ind]
  y = df3['District'][ind]
  z = int(df3['Confirmed'][ind])

  if y == 'kishanganj' and w == 'assam':
    continue  
  if x not in dict1.keys():
    dict4 = {}
    if y in repeat_element:
      key = repeat_dict1[y][w].lower()
      if key not in dict2.keys():
        dict4[key] = z
        dict2[key] = z
          
      elif dict2[key] > z:
        dict4[key] = 0
      else:
        dict4[key] = z - dict2[key]
      
      if dict4[y]<0:
        print(x)
        print(y)
        print(dict4[y])
        print('error1')
        break
      dict2[key] += dict4[key]  

    elif y not in dict2.keys():
      dict4[y] = z
      dict2[y] = z
    else:

      if dict2[y] > z:
        dict4[y] = 0
      else:
        dict4[y] = z - dict2[y]
      
      if dict4[y]<0:
        print(x)
        print(y)
        print(dict4[y])
        print('error1')
        break
      dict2[y] += dict4[y]
    dict1[x] = dict4 

  else:

    if y in repeat_element:
      key = repeat_dict1[y][w].lower()
      if key not in dict2.keys():
        dict1[x][key] = z
        dict2[key] = z
          
      elif dict2[key] > z:
        dict1[x][key] = 0
      else:
        dict1[x][key] = z - dict2[key]
      
      if dict1[x][key]<0:
        print(x)
        print(y)
        print(dict1[x][key])
        print('error1')
        break
      dict2[key] += dict1[x][key]  

    elif y not in dict2.keys():
      dict1[x][y] = z
      dict2[y] = z
    else:
      if dict2[y] > z:
        dict1[x][y] = 0
      else:
        dict1[x][y] = z - dict2[y]

      if dict1[x][y]<0:
        print(x)
        print(y)
        print(z)
        print(dict2[y])
        print(dict1[x][y])
        print('error2')
        break
      dict2[y] += dict1[x][y]
  

print(dict1.keys())
#print(dict1[datetime.datetime(2020,5,1)]) 




dict_keys([Timestamp('2020-03-15 00:00:00'), Timestamp('2020-03-16 00:00:00'), Timestamp('2020-03-17 00:00:00'), Timestamp('2020-03-18 00:00:00'), Timestamp('2020-03-19 00:00:00'), Timestamp('2020-03-20 00:00:00'), Timestamp('2020-03-21 00:00:00'), Timestamp('2020-03-22 00:00:00'), Timestamp('2020-03-23 00:00:00'), Timestamp('2020-03-24 00:00:00'), Timestamp('2020-03-25 00:00:00'), Timestamp('2020-03-26 00:00:00'), Timestamp('2020-03-27 00:00:00'), Timestamp('2020-03-28 00:00:00'), Timestamp('2020-03-29 00:00:00'), Timestamp('2020-03-30 00:00:00'), Timestamp('2020-03-31 00:00:00'), Timestamp('2020-04-01 00:00:00'), Timestamp('2020-04-02 00:00:00'), Timestamp('2020-04-03 00:00:00'), Timestamp('2020-04-04 00:00:00'), Timestamp('2020-04-05 00:00:00'), Timestamp('2020-04-06 00:00:00'), Timestamp('2020-04-07 00:00:00'), Timestamp('2020-04-08 00:00:00'), Timestamp('2020-04-09 00:00:00'), Timestamp('2020-04-10 00:00:00'), Timestamp('2020-04-11 00:00:00'), Timestamp('2020-04-12 00:00:00'), Tim

**Generating cases 26th April onwards from .jsons**

In [ ]:
d1 = datetime.datetime(2020,10,5)
for counter in list(range(3,15)):
  f = open('../aim5/Aim5_Data/aim5_jsons/raw_data{c}.json'.format(c = counter))
  data1 = json.load(f)


  for i in data1['raw_data']:
    a = i['detecteddistrict'].lower()
    b = i['detectedcity'].lower()
    c = i['detectedstate'].lower()
    numcases = i['numcases']
    val = i['dateannounced']
    if val == '' or numcases=='' or i['currentstatus'] !='Hospitalized':
      continue
    da = datetime.datetime.strptime(val,'%d/%m/%Y')
    if da > d1:
      continue
    numcases = int(numcases)
    if a!='' and  a in json_list_neighbor_map.keys() and a not in check :

      #print(da)
      #print(a)
      if  da not in dict1.keys():
        dict4 = {}
        if a in repeat_element and c!='':     #Working for same name districts
          key = repeat_dict1[a][c].lower()
          dict4[key] = numcases
        elif a not in repeat_element:
          dict4[a] = numcases
        dict1[da] = dict4
      
      else:
        if a in repeat_element and c!='':     #Working for same name districts
          key = repeat_dict1[a][c].lower()
          if key not in dict1[da].keys():
            dict1[da][key]=numcases
          else:
            dict1[da][key] = dict1[da][key]+numcases

        elif a not in dict1[da].keys() and a not in repeat_element:
          dict1[da][a]=numcases
        else:
          dict1[da][a] = dict1[da][a]+numcases
    elif b != '' and b in json_list_neighbor_map.keys() and b not in check:

      #print('reached here1')
      #print(b)
      if  da not in dict1.keys():
        dict4 = {}
        if b in repeat_element and c!='':     #Working for same name districts
          key = repeat_dict1[b][c].lower()
          dict4[key] = numcases

        elif b not in repeat_element:
          dict4[b] = numcases
        dict1[da] = dict4

      else:
        if b in repeat_element and c!='':     #Working for same name districts
          key = repeat_dict1[b][c].lower()
          if key not in dict1[da].keys():
            dict1[da][key]=numcases
          else:
            dict1[da][key] = dict1[da][key]+numcases

        elif b not in dict1[da].keys() and b not in repeat_element:
          dict1[da][b]=numcases
        else:
          dict1[da][b] = dict1[da][b]+numcases

print(dict1.keys())


dict_keys([Timestamp('2020-03-15 00:00:00'), Timestamp('2020-03-16 00:00:00'), Timestamp('2020-03-17 00:00:00'), Timestamp('2020-03-18 00:00:00'), Timestamp('2020-03-19 00:00:00'), Timestamp('2020-03-20 00:00:00'), Timestamp('2020-03-21 00:00:00'), Timestamp('2020-03-22 00:00:00'), Timestamp('2020-03-23 00:00:00'), Timestamp('2020-03-24 00:00:00'), Timestamp('2020-03-25 00:00:00'), Timestamp('2020-03-26 00:00:00'), Timestamp('2020-03-27 00:00:00'), Timestamp('2020-03-28 00:00:00'), Timestamp('2020-03-29 00:00:00'), Timestamp('2020-03-30 00:00:00'), Timestamp('2020-03-31 00:00:00'), Timestamp('2020-04-01 00:00:00'), Timestamp('2020-04-02 00:00:00'), Timestamp('2020-04-03 00:00:00'), Timestamp('2020-04-04 00:00:00'), Timestamp('2020-04-05 00:00:00'), Timestamp('2020-04-06 00:00:00'), Timestamp('2020-04-07 00:00:00'), Timestamp('2020-04-08 00:00:00'), Timestamp('2020-04-09 00:00:00'), Timestamp('2020-04-10 00:00:00'), Timestamp('2020-04-11 00:00:00'), Timestamp('2020-04-12 00:00:00'), Tim

In [ ]:
serialized_d = {str(k): v for k, v in dict1.items()}

with open('../aim5/Aim5_Data/processed_data.json','w') as f:
  json.dump(serialized_d,f,indent=3)